Для сравнения в этом домашнем задании я выбрала pymorphy2, natasha, stanza. Два последних используют universal pos tags, у pymorphy2 чуть другие теги, что уравнивается в коде.

Для корпуса брала предложения из разных источников, среди которых были:

- статьи о моде 
- отрывки из стихотворений с окказионализмами
- предложения с информацией о различных учреждениях, записанных аббревиатурами или сокращениями

В первом случае интерес представляли заимствованные слова-неологизмы, которых ещё, возможно, нет в словарях, а также слова, написанные через дефис, которые теггеры могли бы разделить, а могли бы и нет (бьюти-тренды, ультра-клеш и т.п.)
Во втором были неоднозначными сами авторские окказионализмы, также отсутствующие в словарях. И в последнем случае трудности для теггинга должны были представить сами аббревиатуры и разнообразные сокращения.
Также в отобранных предложенияхх встречались имена и фамилии, в том числе иностранные.

In [1]:
import stanza
from natasha import Doc, MorphVocab, NewsEmbedding, NewsMorphTagger, Segmenter
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize

In [3]:
# загрузка моделек
stanza_pipeline = stanza.Pipeline("ru")
segmenter = Segmenter()
natasha_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph = MorphAnalyzer()

2023-10-07 22:50:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-07 22:58:17 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2023-10-07 22:58:17 INFO: Using device: cpu
2023-10-07 22:58:17 INFO: Loading: tokenize
2023-10-07 22:58:18 INFO: Loading: pos
2023-10-07 22:58:19 INFO: Loading: lemma
2023-10-07 22:58:19 INFO: Loading: depparse
2023-10-07 22:58:20 INFO: Loading: ner
2023-10-07 22:58:22 INFO: Done loading processors!


In [5]:
# чтение корпуса из файла и заодно проверка количества слов
with open("corpuss.txt", "r", encoding = "utf-8") as file:
    text = file.read()
    print(len(text.split()))
    text = text.split("\n")

232


In [6]:
#Stanza
stanza = []
for line in text:
    stanza_doc = stanza_pipeline(line)
    stanza_dict = {}
    for sent in stanza_doc.sentences:
        for word in sent.words:
            if word.upos != "PUNCT":
                stanza_dict[word.text] = word.upos
    for key, value in stanza_dict.items():
        if value == "SCONJ" or value == "CCONJ":
            stanza_dict[key] = "CONJ"
        elif value == "PROPN":
            stanza_dict[key] = "NOUN"
        elif value == "AUX":
            stanza_dict[key] = "VERB"
    stanza.append(stanza_dict)

In [7]:
#Natasha
natasha = []
for line in text:
    natasha_doc = Doc(line)
    natasha_doc.segment(segmenter)
    natasha_doc.tag_morph(morph_tagger)
    natasha_dict = {}
    for token in natasha_doc.tokens:
        if token.pos != "PUNCT":
            natasha_dict[token.text] = token.pos
    for key, value in natasha_dict.items():
        if value == "SCONJ" or value == "CCONJ":
            natasha_dict[key] = "CONJ"
        elif value == "PROPN":
            natasha_dict[key] = "NOUN"
        elif value == "AUX":
            natasha_dict[key] = "VERB"
    natasha.append(natasha_dict)

In [17]:
#pymorphy2
py = []
for line in text:
    tokens = word_tokenize(line)
    py_dict = {}
    for token in tokens:
        analysis = morph.parse(token)[0]
        if str(analysis.tag).split(",")[0] != "PNCT":
            py_dict[token] = str(analysis.tag).split(",")[0].split()[0]
    for key, value in py_dict.items():
        if value == "ADVB" or value == "PRED":
            py_dict[key] = "ADV"
        elif value == "ADJF":
            py_dict[key] = "ADJ"
        elif value == "INFN":
            py_dict[key] = "VERB"
        elif value == "COMP":
            py_dict[key] = "ADJ"
        elif value == "NPRO":
            py_dict[key] = "PRON"
        elif value == "PREP":
            py_dict[key] = "ADP"
        elif value == "PRTF" or value == "PRTS":
            py_dict[key] = "VERB"
        elif value == "PRCL":
            py_dict[key] = "PART"
    py.append(py_dict)

Посмотрим, что получилось

In [12]:
print("Stanza:")
for d in stanza[:5]:
    for word, pos in d.items():
        print(f"{word}: {pos}")
    print("\n")

Stanza:
Специально: ADV
для: ADP
вас: PRON
мы: PRON
собрали: VERB
большой: ADJ
шопинг: NOUN
гид: NOUN
по: ADP
осенним: ADJ
трендам: NOUN
чтобы: CONJ
подготовка: NOUN
к: ADP
новому: ADJ
сезону: NOUN
не: PART
вызывала: VERB
негативных: ADJ
эмоций: NOUN
и: CONJ
проходила: VERB
легко: ADV


Среди: ADP
главных: ADJ
трендов: NOUN
оказались: VERB
традиционно: ADV
мужские: ADJ
туфли: NOUN
оксфорды: NOUN
броги: NOUN
и: CONJ
монки: NOUN


Платья: NOUN
с: ADP
бахромой: NOUN
и: CONJ
драпировками: NOUN
металлик: NOUN
брюки: NOUN
карго: NOUN
все: DET
это: PRON
стабильно: ADV
не: PART
покидает: VERB
модную: ADJ
арену: NOUN


Причем: CONJ
речь: NOUN
идет: VERB
не: PART
о: ADP
легком: ADJ
клеше: NOUN
который: PRON
мы: PRON
привыкли: VERB
видеть: VERB
на: ADP
инфлюэнсерах: NOUN
в: ADP
последние: ADJ
годы: NOUN
а: CONJ
настоящий: ADJ
ультра: NOUN
клеш: NOUN
с: ADP
расширенными: VERB
книзу: NOUN
штанинами: NOUN
до: ADP
такой: DET
степени: NOUN
что: CONJ
них: PRON
можно: ADV
запутаться: VERB


Она: PRON
не

In [14]:
print("Natasha:")
for d in natasha[:5]:
    for word, pos in d.items():
        print(f"{word}: {pos}")
    print("\n")

Natasha:
Специально: ADV
для: ADP
вас: PRON
мы: PRON
собрали: VERB
большой: ADJ
шопинг-гид: NOUN
по: ADP
осенним: ADJ
трендам: NOUN
чтобы: CONJ
подготовка: NOUN
к: ADP
новому: ADJ
сезону: NOUN
не: PART
вызывала: VERB
негативных: ADJ
эмоций: NOUN
и: CONJ
проходила: VERB
легко: ADV


Среди: ADP
главных: ADJ
трендов: NOUN
оказались: VERB
традиционно: ADV
мужские: ADJ
туфли: NOUN
оксфорды: NOUN
броги: NOUN
и: CONJ
монки: NOUN


Платья: NOUN
с: ADP
бахромой: NOUN
и: CONJ
драпировками: NOUN
металлик: NOUN
брюки: NOUN
карго: NOUN
все: DET
это: PRON
стабильно: ADV
не: PART
покидает: VERB
модную: ADJ
арену: NOUN


Причем: CONJ
речь: NOUN
идет: VERB
не: PART
о: ADP
легком: ADJ
клеше: NOUN
который: PRON
мы: PRON
привыкли: VERB
видеть: VERB
на: ADP
инфлюэнсерах: NOUN
в: ADP
последние: ADJ
годы: NOUN
а: CONJ
настоящий: ADJ
ультра-клеш: NOUN
с: ADP
расширенными: VERB
книзу: NOUN
штанинами: NOUN
до: ADP
такой: DET
степени: NOUN
что: CONJ
них: PRON
можно: ADV
запутаться: VERB


Она: PRON
не: PART
расс

In [19]:
print("Pymorphy:")
for d in py[:5]:
    for word, pos in d.items():
        print(f"{word}: {pos}")
    print("\n")

Pymorphy:
Специально: ADV
для: ADP
вас: PRON
мы: PRON
собрали: VERB
большой: ADJ
шопинг-гид: NOUN
по: ADP
осенним: ADJ
трендам: NOUN
чтобы: CONJ
подготовка: NOUN
к: ADP
новому: ADJ
сезону: NOUN
не: PART
вызывала: VERB
негативных: ADJ
эмоций: NOUN
и: CONJ
проходила: VERB
легко: ADV


Среди: ADP
главных: ADJ
трендов: NOUN
оказались: VERB
традиционно: ADV
мужские: ADJ
туфли: NOUN
оксфорды: NOUN
броги: ADJS
и: CONJ
монки: NOUN


Платья: NOUN
с: ADP
бахромой: NOUN
и: CONJ
драпировками: NOUN
металлик: NOUN
брюки: NOUN
карго: NOUN
все: PART
это: PART
стабильно: ADV
не: PART
покидает: VERB
модную: ADJ
арену: NOUN


Причем: CONJ
речь: NOUN
идет: VERB
не: PART
о: ADP
легком: ADJ
клеше: NOUN
который: ADJ
мы: PRON
привыкли: VERB
видеть: VERB
на: ADP
инфлюэнсерах: NOUN
в: ADP
последние: ADJ
годы: NOUN
а: CONJ
настоящий: ADJ
ультра-клеш: NOUN
с: ADP
расширенными: VERB
книзу: ADV
штанинами: NOUN
до: ADP
такой: ADJ
степени: NOUN
что: CONJ
них: PRON
можно: ADV
запутаться: VERB


Она: PRON
не: PART
расс

In [16]:
#достаем нашу разметку
answers = []
with open("answers.txt", "r", encoding = "utf-8") as file:
    sents = file.read().split("\n\n\n")
    for s in sents:
        d = {}
        pairs = s.split("\n")
        for pair in pairs:
            d[pair.split()[0]] = pair.split()[1]
        answers.append(d)

In [21]:
#так будем считать, что лучше
def accuracy(ans, pred):
    sum = 0
    score = 0
    for i in range(0, len(ans)):
        for word in ans[i]:
            sum += 1
            if word in pred[i]:
                if ans[i][word] == pred[i][word]:
                    score += 1
    return score/sum

In [132]:
accuracy(answers, stanza)

0.9155555555555556

In [131]:
accuracy(answers, natasha)

0.8933333333333333

In [22]:
accuracy(answers, py)

0.8844444444444445

Лучше всех справилась stanza!

N-gramm-ы, которые могли бы помочь в прошлой домашке (на большее меня не хватило sorry i'm weak like that):
1. самая база, про которую я писала и в прошлой домашке: *не* + глагол / *не* + прилагательное, меняющее окраску глагола/прилагательного на противоположное значение (не стала разделять на два пункта глагол и прилагательное, потому что суть в целом одна).
2. *до* + существительное + *далеко*. Часто могут использоваться фразы типа *до идеала далеко*, *до уровня далеко*, и в контексте такой фразы само по себе положительное слово (напр. *идеал*) уже не несет в себе положительной окраски, а выступает средством контраста
3. *якобы* + прилагательное. Тут похожая ситуация с *не*: употребление такого сочетания может превратить какое-либо положительное по окраске прилагательное в средство невыгодного сравнения: *якобы престижный*, *якобы интересный*, то есть общая эмоциональная окраска биграммы будет уже негативной.